**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Linux namespaces

![Namespaces](namespaces.png)

![container vs hypervisor](docker-hypervisor.png)

In [1]:
docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED         STATUS         PORTS     NAMES
658daf1a9ae5   nginx                  "/docker-entrypoint.…"   2 minutes ago   Up 2 minutes             k8s_my-nginx_my-nginx-5b56ccd65f-5tbgs_default_1ad19392-eb5c-4df9-aa54-efd0ecfe0e2c_0
18ea1f6cc811   k8s.gcr.io/pause:3.5   "/pause"                 2 minutes ago   Up 2 minutes             k8s_POD_my-nginx-5b56ccd65f-5tbgs_default_1ad19392-eb5c-4df9-aa54-efd0ecfe0e2c_0
095296be488c   nginx                  "/docker-entrypoint.…"   2 hours ago     Up 2 hours               k8s_my-nginx_my-nginx-5b56ccd65f-lrbr4_default_36d53d62-dccb-45ac-8f0d-6f850d92d8b1_0
fdac76cbe36e   nginx                  "/docker-entrypoint.…"   2 hours ago     Up 2 hours               k8s_my-nginx_my-nginx-5b56ccd65f-nxbdq_default_f99944db-1790-473b-995c-7faad2371330_0
4dd8cd66fc29   k8s.gcr.io/pause:3.5   "/pause"                 2 hours ago     Up 2 hours               k8s_POD_my-nginx-5b56ccd65f-lrb

In [2]:
crictl ps

CONTAINER           IMAGE                                                                           CREATED             STATE               NAME                      ATTEMPT             POD ID
658daf1a9ae5c       nginx@sha256:9522864dd661dcadfd9958f9e0de192a1fdda2c162a35668ab6ac42b465f0603   2 minutes ago       Running             my-nginx                  0                   18ea1f6cc8119
095296be488cc       nginx@sha256:9522864dd661dcadfd9958f9e0de192a1fdda2c162a35668ab6ac42b465f0603   2 hours ago         Running             my-nginx                  0                   4dd8cd66fc293
fdac76cbe36e6       nginx@sha256:9522864dd661dcadfd9958f9e0de192a1fdda2c162a35668ab6ac42b465f0603   2 hours ago         Running             my-nginx                  0                   72b88b9699279
541f172568236       nginx@sha256:9522864dd661dcadfd9958f9e0de192a1fdda2c162a35668ab6ac42b465f0603   2 hours ago         Running             my-nginx                  0                   d9b11f066ead0
3cbdff5

In [ ]:
kubectl create deployment nginx --image=nginx

In [3]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-5tbgs   1/1     Running   0          3m11s
pod/my-nginx-5b56ccd65f-7br4l   1/1     Running   0          143m
pod/my-nginx-5b56ccd65f-lrbr4   1/1     Running   0          126m
pod/my-nginx-5b56ccd65f-nxbdq   1/1     Running   0          126m
pod/my-nginx-5b56ccd65f-p5k7h   1/1     Running   0          143m

NAME                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
service/kubernetes   ClusterIP      10.96.0.1        <none>           443/TCP        164m
service/my-nginx     LoadBalancer   10.105.208.150   10.105.208.150   80:32335/TCP   143m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   5/5     5            5           143m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   5         5         5       143m


In [4]:
POD=k8s_my-nginx_my-nginx-5b56ccd65f-5tbgs_default_1ad19392-eb5c-4df9-aa54-efd0ecfe0e2c_0

In [5]:
STATE_PID=$(docker inspect --format '{{.State.Pid}}' $POD)
echo Enter
echo nsenter -t $STATE_PID -n bash
echo in a terminal  

Enter
nsenter -t 42132 -n bash
in a terminal


In [6]:
nsenter --help


Usage:
 nsenter [options] [<program> [<argument>...]]

Run a program with namespaces of other processes.

Options:
 -a, --all              enter all namespaces
 -t, --target <pid>     target process to get namespaces from
 -m, --mount[=<file>]   enter mount namespace
 -u, --uts[=<file>]     enter UTS namespace (hostname etc)
 -i, --ipc[=<file>]     enter System V IPC namespace
 -n, --net[=<file>]     enter network namespace
 -p, --pid[=<file>]     enter pid namespace
 -C, --cgroup[=<file>]  enter cgroup namespace
 -U, --user[=<file>]    enter user namespace
 -S, --setuid <uid>     set uid in entered namespace
 -G, --setgid <gid>     set gid in entered namespace
     --preserve-credentials do not touch uids or gids
 -r, --root[=<dir>]     set the root directory
 -w, --wd[=<dir>]       set the working directory
 -F, --no-fork          do not fork before exec'ing <program>
 -Z, --follow-context   set SELinux context according to --target PID

 -h, --help             display this help
 -V

In [7]:
nsenter -t $STATE_PID -n ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
18: eth0@if19: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:06 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.6/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [8]:
ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
3: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:89:7f:62:b9 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
5: vethaf24f96@if4: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 3a:96:c6:33:10:e4 brd ff:ff:ff:ff:ff:ff link-netnsid 1
7: vethba082f8@if6: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue master docker0 state UP group default 
    link/ether 7a:91:42:3e:5c:e6 brd ff:ff:ff:ff:ff:ff link-netnsid 2
9: vethf624b45@if8: <BROAD

 # Danger!
 
 OpenVpn works and could be used as backdoor

In [9]:
nsenter -t $STATE_PID -a hostname

my-nginx-5b56ccd65f-5tbgs


In [10]:
nsenter -t $STATE_PID -a ip a

nsenter: failed to execute ip: No such file or directory


: 127

In [11]:
nsenter -t $STATE_PID -n ip route

default via 172.17.0.1 dev eth0 
172.17.0.0/16 dev eth0 proto kernel scope link src 172.17.0.6 


In [12]:
ip route

default via 192.168.49.1 dev eth0 
172.17.0.0/16 dev docker0 proto kernel scope link src 172.17.0.1 
192.168.49.0/24 dev eth0 proto kernel scope link src 192.168.49.2 


In [13]:
nsenter -t $STATE_PID -a ls

bin   docker-entrypoint.d   home   media  proc	sbin  tmp
boot  docker-entrypoint.sh  lib    mnt	  root	srv   usr
dev   etc		    lib64  opt	  run	sys   var


In [14]:
ls

 AccountWithRBAC.ipynb          PodSecurityPolicyPermissive.ipynb
 AuditLogs.ipynb                PodSecurityPolicyStrict.ipynb
 Calico-Iptables.ipynb         'Postgres Zalando.ipynb'
 ClusterOverview.ipynb          README.md
 ContainerdCloudbomb.ipynb      ServiceAccountToken.ipynb
 CouchDB.ipynb                  Unprivileged.ipynb
 CrioBomb.ipynb                 Untitled.ipynb
 Harbor.ipynb                   audit-policy.yaml
 Helm.ipynb                     cloudbomb
 Ingress.ipynb                  default-rbac.png
 IngressKong.ipynb              defaut-rbac.dot
 IngressKongStart.ipynb         docker-hypervisor.png
 IngressStart.ipynb             istio-1.12.0
 Istio-Traffic-Mgmt.ipynb       kube-bench
 Istio1.8.2.ipynb               kube-bench.ipynb
 IstioHack.ipynb                minikube.dot
 Kube-Scan-Cloudbomb.ipynb      minikube.png
 KubernetesBasics.ipynb         my-kubeconfig
 License.ipynb                  my-kubeconfig.full.tmp
'Linux Namespaces.ipynb'        my-kubeconfig.t

In [15]:
nsenter -t $STATE_PID -a ps auxwww

nsenter: failed to execute ps: No such file or directory


: 127

In [16]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  21860 11512 ?        Ss   10:37   0:01 /sbin/init
root         175  0.0  0.0  27116 11504 ?        S<s  10:37   0:00 /lib/systemd/systemd-journald
message+     187  0.0  0.0   7020  3588 ?        Ss   10:37   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         193  0.1  0.2 1493532 47208 ?       Ssl  10:37   0:13 /usr/bin/containerd
root         215  0.0  0.0  12184  7324 ?        Ss   10:37   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         220  0.5  0.5 343180 87752 ?        Rsl  10:37   1:01 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         463  3.0  0.6 1827328 98916 ?       Ssl  10:37   5:12 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [17]:
nsenter -t $STATE_PID -p ps auxwww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  21860 11512 ?        Ss   10:37   0:01 /sbin/init
root         175  0.0  0.0  27116 11504 ?        S<s  10:37   0:00 /lib/systemd/systemd-journald
message+     187  0.0  0.0   7020  3588 ?        Ss   10:37   0:00 /usr/bin/dbus-daemon --system --address=systemd: --nofork --nopidfile --systemd-activation --syslog-only
root         193  0.1  0.2 1493532 47208 ?       Ssl  10:37   0:13 /usr/bin/containerd
root         215  0.0  0.0  12184  7324 ?        Ss   10:37   0:00 sshd: /usr/sbin/sshd -D [listener] 0 of 10-100 startups
root         220  0.5  0.5 343180 87752 ?        Rsl  10:37   1:01 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root         463  3.0  0.6 1827328 98916 ?       Ssl  10:37   5:12 /usr/bin/dockerd -H tcp://0.0.0.0:2376 -H unix:///var/run/docker.sock --default-ulimit=nofile=1048576:1048576 --tlsverify --tlscacert /etc/doc

In [18]:
ls /proc/$STATE_PID

arch_status         cwd        maps           pagemap       stack
attr                environ    mem            patch_state   stat
autogroup           exe        mountinfo      personality   statm
auxv                fd         mounts         projid_map    status
cgroup              fdinfo     mountstats     root          syscall
clear_refs          gid_map    net            sched         task
cmdline             io         ns             schedstat     timens_offsets
comm                latency    numa_maps      sessionid     timers
coredump_filter     limits     oom_adj        setgroups     timerslack_ns
cpu_resctrl_groups  loginuid   oom_score      smaps         uid_map
cpuset              map_files  oom_score_adj  smaps_rollup  wchan
